# COSC 4610/5610 Project 1 (spring 2022)

**Authors**: Carl Barcenas

**Emails**: carlanthony.barcenas@marquette.edu

**Submission.** Please insert your names and emails above, save your code in this notebook, and explain what you are doing along with your findings in text cells. You can think of it as a technical report with code. Before submission, please use `Kernel -> Restart & Run All` in the Jupyter menu to verify your code is runnable and save all outputs. Afterwards, you can either upload your raw notebook (`prj1.ipynb`) or an exported PDF version. 


In this project, we will practice with data cleaning and transformation methods on the California house sales data and check their effects on the price prediction model. The model training and the evaluation code will be given. Your job is to try different data pre-processing methods to obtain the best root mean squared logarithmic error (RMSLE) on the test dataset.

## Set up the environment
We will use AutoGluon (https://auto.gluon.ai/) to simplify the modeling step. However, AutoGluon works only on Python 3.7 so far. It also only works on Linux and Mac. Thus, it's necessary to set up the environment first. 

### Python 3.7 on Ubuntu (and WSL)
If you need to install Python 3.7 in Ubuntu (and WSL), you can follow the steps
1. Start by updating the packages list and installing the prerequisites:

`sudo apt update`

`sudo apt install software-properties-common`

2. Next, add the deadsnakes PPA to your sources list:

`sudo add-apt-repository ppa:deadsnakes/ppa`

When prompted press Enter to continue:

Press [ENTER] to continue or Ctrl-c to cancel adding it.Copy

3. Once the repository is enabled, install Python 3.7 with:

`sudo apt install python3.7`

If you are using Windows, you want to set up the WSL first (https://docs.microsoft.com/en-us/windows/wsl/install), install Python 3.7, and then run the jupyter server with the command `jupyter notebook --no-browser`. Virtualenv is also recommended if the default Python version is not 3.7. You can then access the server in your Windows browser with prompted URL (check the line after "Or copy and paste one of these URLs"). 

### Python 3.7 on Mac
Run the command `brew install python@3.7`. The binary should be on /usr/local/opt/python@3.7/bin/python3.7


### Virtualenv for multiple versions of Python
If you are using Linux (or WSL) and Mac but not on Python 3.7 (e.g., your system has a higher version Python), the following method allows you to set up an isolated virtual environment, which does not interfere with your existing Python setups. 
1. download and install Python 3.7. Get the path to the specific Python3 binary. For example, in my system, it's installed at /usr/local/opt/python@3.7/bin/python3

2. run `pip3 install virtualenv` to install the tool virtualenv

3. run `virtualenv -p path_to_your_python3.7_binary py37` to create the virtual environment named py37. You will see a subdirectory named py37 is created under the current directory.

4. run `source py37/bin/activate` to activate the py37 environment. You will see (py37) shows up in your command line prompt. You can run `deactivate` later to exit the virtual environment

5. after py37 is activiated, install the necessary packages: `pip3 install notebook numpy pandas autogluon mxnet`

6. run `jupyter notebook` and you are ready to use the python 3.7 environment and the autogluon package.




## Prepare Data 

Download the data from http://www.cs.mu.edu/~keke/dm/data/house_sales.ftr. Note that we use the [`feather` format](https://arrow.apache.org/docs/python/feather.html), which is faster to read than CSV but uses more disk space. 
The following code needs at least 2GB memory. If using a local runtime, please make sure your machine has enough memory. 

In [1]:
# If the pacakges are not installed, run the following line once to install. You may need to restart your runtime afterwards:
#!pip3 install numpy pandas autogluon mxnet --upgrade  
import pandas as pd
import numpy as np

data = pd.read_feather('house_sales.ftr')

We select a few columns to demonstrate. You need to select more columns to make your model more accurate. 

In [2]:
df = data[['Sold Price', 'Sold On', 'Type', 'Year built', 'Bedrooms', 'Bathrooms']].copy()
# uncomment the below line to save memory
# del data
df.dtypes

Sold Price    object
Sold On       object
Type          object
Year built    object
Bedrooms      object
Bathrooms     object
dtype: object

We copy the code from EDA to convert `Sold Price` to numerical values, which is our prediction target. We also remove examples whose prices are too high or too low. For more examples, you can revisit the notebook https://www.cs.mu.edu/~keke/dm/house.ipynb.

In [3]:

c = 'Sold Price'
if c in df.select_dtypes('object').columns:
    df.loc[:,c] = np.log10(
            pd.to_numeric(df[c].replace(r'[$,-]', '', regex=True)) + 1)
    
df = df[(df['Sold Price'] >= 4 ) & (df['Sold Price'] <= 8 )] # prices between 10^4 and 10^8


We use the house sales between 2021-2-15 and 2021-3-1 as our test data. You can use any example before 2021-2-15 for training, but not after. In other words, we pretend we are launching our model on 2021-2-15 and testing it for 2 weeks. Here we only use sales in 2021 for fast training, but you can use more to improve accuracy. 

In [4]:
test_start, test_end = pd.Timestamp(2021, 2, 15), pd.Timestamp(2021, 3, 1)
train_start = pd.Timestamp(2021, 1, 1) # you can change the start time stamp to include more training 
df['Sold On'] = pd.to_datetime(df['Sold On'], errors='coerce')
train = df[(df['Sold On'] >= train_start) & (df['Sold On'] < test_start)]
test = df[(df['Sold On'] >= test_start) & (df['Sold On'] < test_end)]
train.shape, test.shape


((24872, 6), (11510, 6))

Define our evaluation metric. 

In [5]:
def rmsle(y_hat, y):
    # we already used log prices before, so we only need to compute RMSE
    return sum((y_hat - y)**2 / len(y))**0.5

## Regression Baseline

We compute the baseline model with a gradient boosting regressor (GBM). You are welcome to try other models to achieve the best result. Available options include ‘GBM’ (LightGBM), ‘CAT’ (CatBoost), ‘XGB’ (XGBoost), ‘RF’ (random forest), ‘XT’ (extremely randomized trees), ‘KNN’ (k-nearest neighbors), ‘LR’ (linear regression), ‘NN’ (neural network with MXNet backend), ‘FASTAI’ (neural network with FastAI backend). However, we recommend that you reuse the following training code so that you can focus on data preprocessing.

In [6]:
from autogluon.tabular import TabularPredictor

label = 'Sold Price'    
predictor = TabularPredictor(label=label).fit(train, hyperparameters={'GBM':{}})


No path specified. Models will be saved in: "AutogluonModels/ag-20220517_030223\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220517_030223\"
AutoGluon Version:  0.4.0
Python Version:     3.9.12
Operating System:   Windows
Train Data Rows:    24872
Train Data Columns: 5
Label Column: Sold Price
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (7.546542675816042, 4.000043427276863, 5.75084, 0.39719)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    4016.99 MB
	Train Data (Original)  Memory Usage: 6.15 MB (0.2% of avail

Next, we compute the importance of each feature, along with several other metrics. It loooks like the `Sold On` feature is not very useful, likely because the houses in the test data were all sold late. You can choose to either remove such a feature, or find a way to extract a more useful presentation from it.

In [7]:
predictor.feature_importance(test)

Computing feature importance via permutation shuffling for 5 features using 1000 rows with 3 shuffle sets...
	1.3s	= Expected runtime (0.43s per shuffle set)
	0.72s	= Actual runtime (Completed 3 of 3 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
Type,0.097894,0.008260,0.001182,3,0.145224,0.050564
Bathrooms,0.091230,0.010910,0.002366,3,0.153743,0.028716
Year built,0.059341,0.010620,0.005254,3,0.120194,-0.001512
Bedrooms,0.012770,0.003754,0.013809,3,0.034282,-0.008741
Sold On,0.000288,0.000944,0.324912,3,0.005695,-0.005119


Finally, let's predict and evaluate the RMSLE. The smaller the RMSLE value, the better. 

In [8]:
#test
preds = predictor.predict(test.drop(columns=[label]))
rmsle(preds, test[label])

0.2682227238087204

Put the evaluation part in a function for easier use. You may modify it as you like. 

In [9]:
def eval_model(train, test):
    from autogluon.tabular import TabularPredictor
    label = 'Sold Price'    
    predictor = TabularPredictor(label=label).fit(train, hyperparameters={'GBM':{}})
    preds = predictor.predict(test.drop(columns=[label]))
    print(rmsle(preds, test[label]))

# with updated train/test datasets, call the function eval_model(train, test)
#eval_model(train, test)


## Your Tasks

Your goal is to train a model using the features in the original dataset that minimizes the RMSLE on the validation dataset. While the naïve model achieves an RMSLE of ~0.3, it is possible to achieve an RMSLE of less than 0.08 on the same dataset (extra credit: +5, if you can find a model with RMSLE <0.1 :-) ).  

Complete the following tasks. 

1. Try more features: We only selected a small set of columns to use in training. You can add more, especially the ones we examined in EDA. You can add batches of features progressively. Report what you have done and what you have observed about the RMSLE change. 

2. Data type conversion: Most data columns are strings; you may need to convert them into numerical values. Describe how you convert them, and report the RMSLE change.

3. Data cleaning: There are NAN and outliers sprinkled throughout the dataset. You should find ways to selectively filter and remove them. Describe what you do and report the RMSLE change.

4. More examples: We only included sales made in 2021; there is a large number of examples in previous years that you can also include. You may try different strategies to include more training data. However, alway keep the house sales between 2021-2-15 and 2021-3-1 as our test data. Describe what you do and report the RMSLE change. 

You may reorganize the code with functions to reuse pieces of code and make your code clearer. Report any RMSLE changes (may increase or decrease after a specific experiment).


# STUDENT SOLUTION SECTION
## Evaluating the Data
If I want to add features, I need to know what I'm working with. I want to see the possible columns I can use and other data.

In [10]:
# Placing this here so I don't have to rerun from the top.
import pandas as pd
import numpy as np

def rmsle(y_hat, y):
    # we already used log prices before, so we only need to compute RMSE
    return sum((y_hat - y)**2 / len(y))**0.5

def eval_model(train, test):
    from autogluon.tabular import TabularPredictor
    label = 'Sold Price'    
    predictor = TabularPredictor(label=label).fit(train, hyperparameters={'GBM':{}})
    preds = predictor.predict(test.drop(columns=[label]))
    print(preds)
    print(rmsle(preds, test[label]))

data = pd.read_feather('house_sales.ftr')

# Visualizing the data
display(data)

,Id,Address,Sold Price,Sold On,Summary,Type,Year built,Heating,Cooling,Parking,...,Well Disclosure,remodeled,DOH2,SerialX,Full Baths,Tax Legal Lot Number,Tax Legal Block Number,Tax Legal Tract Number,Building Name,Zip
0,2080183300,"11205 Monterey,","$2,000,000",01/31/20,"11205 Monterey, San Martin, CA 95046 is a sing...",SingleFamily,No Data,No Data,No Data,0 spaces,...,None,None,None,None,None,None,None,None,None,95046
1,20926300,"5281 Castle Rd,","$2,100,000",02/25/21,Spectacular Mountain and incredible L.A. City ...,SingleFamily,1951,Central,"Central Air, Dual","Driveway, Driveway - Brick",...,None,None,None,None,None,None,None,None,None,91011
2,19595300,"3581 Butcher Dr,","$1,125,000",11/06/19,Eichler Style home! with Santa Clara High! in ...,SingleFamily,1954,Central Forced Air - Gas,Central AC,"Garage, Garage - Attached, Covered",...,None,None,None,None,None,None,None,None,None,95051
3,300472200,"2021 N Milpitas Blvd,","$36,250,000",10/02/20,"2021 N Milpitas Blvd, Milpitas, CA 95035 is a ...",Apartment,1989,Other,No Data,"Mixed, Covered",...,None,None,None,None,None,None,None,None,None,95035
4,2074492000,"LOT 4 Tool Box Spring Rd,","$140,000",10/19/20,Beautiful level lot dotted with pine trees ro...,VacantLand,No Data,No Data,No Data,0 spaces,...,None,None,None,None,None,None,None,None,None,92561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164939,72555199,"88 Lakeshore Ct,","$563,527",02/19/21,STUNNING LAKE VIEW AND BAY VIEW! Exquisite Res...,Condo,1991,"Baseboard, Electric",None,"Carport, Covered, Guest",...,None,None,None,None,None,None,None,None,None,94804
164940,94643599,"3785 Wilshire Blvd PENTHOUSE 3,","$2,650,000",05/31/18,Arguably the best unit at Solair. This top flo...,Condo,2009,Central,Central,Covered,...,None,None,None,None,None,None,None,None,None,90010
164941,300479799,"312 Circuit Way,","$1,357,000",11/04/19,"312 Circuit Way, Mountain View, CA 94043 is a ...",Condo,2019,Other,No Data,"Garage, Garage - Attached, Covered",...,None,None,None,None,None,None,None,None,None,94043
164942,15504399,"2 Cape Breton Ct,","$1,400,000",07/23/20,Park Pacifica CHECK OUT THE VIDEO Entering Ca...,SingleFamily,1973,Central Forced Air - Gas,None,"Underground/Basement, Garage - Attached",...,None,None,None,None,None,None,None,None,None,94044


In [14]:
# Full column list
#print(data.columns.tolist())

#display(data['Type'])
#display(data['Type'].value_counts())
#data['Type'].isna().sum()

## 1.1 Try Adding New Features
In particular, I wanted to add Heating, Cooling, and Parking into the equation as I think these are reasonable things to consider when purchasing a new house/apartment that will drastically affect the cost of the housing.

In [12]:
# Redefine df
df = data[['Sold Price', 'Sold On', 'Type', 'Year built', 'Bedrooms', 'Bathrooms', 
           'Heating', 'Cooling', 'Parking']].copy()

# Establish prediction target and Convert sold price to numerical data.
c = 'Sold Price'
if c in df.select_dtypes('object').columns:
    df.loc[:,c] = np.log10(
            pd.to_numeric(df[c].replace(r'[$,-]', '', regex=True)) + 1)

# Removing outliers in sold price
df = df[(df['Sold Price'] >= 4 ) & (df['Sold Price'] <= 8 )] # prices between 10^4 and 10^8

# Get train, test sets
test_start, test_end = pd.Timestamp(2021, 2, 15), pd.Timestamp(2021, 3, 1)
train_start = pd.Timestamp(2021, 1, 1) # you can change the start time stamp to include more training 
df['Sold On'] = pd.to_datetime(df['Sold On'], errors='coerce')
train = df[(df['Sold On'] >= train_start) & (df['Sold On'] < test_start)]
test = df[(df['Sold On'] >= test_start) & (df['Sold On'] < test_end)]
train.shape, test.shape

# Evaluate Model
eval_model(train, test)

No path specified. Models will be saved in: "AutogluonModels/ag-20220517_030350\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220517_030350\"
AutoGluon Version:  0.4.0
Python Version:     3.9.12
Operating System:   Windows
Train Data Rows:    24872
Train Data Columns: 8
Label Column: Sold Price
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (7.546542675816042, 4.000043427276863, 5.75084, 0.39719)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    3792.29 MB
	Train Data (Original)  Memory Usage: 11.59 MB (0.3% of avai

1         6.165126
5         5.895925
40        6.012253
53        5.208252
58        5.954611
            ...   
164894    6.080437
164897    6.006753
164920    6.353038
164932    5.572160
164939    5.669645
Name: Sold Price, Length: 11510, dtype: float32
0.24332273066046678


In [15]:
print(preds)

# Check the importance of the new column additions.
predictor.feature_importance(test)

These features in provided data are not utilized by the predictor and will be ignored: ['Heating', 'Cooling', 'Parking']
Computing feature importance via permutation shuffling for 5 features using 1000 rows with 3 shuffle sets...
	1.5s	= Expected runtime (0.5s per shuffle set)


1         6.031573
5         5.873736
40        5.777894
53        5.065905
58        5.881350
            ...   
164894    6.175815
164897    5.858469
164920    6.214445
164932    5.671451
164939    5.663757
Name: Sold Price, Length: 11510, dtype: float32


	0.85s	= Actual runtime (Completed 3 of 3 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
Type,0.097894,0.008260,0.001182,3,0.145224,0.050564
Bathrooms,0.091230,0.010910,0.002366,3,0.153743,0.028716
Year built,0.059341,0.010620,0.005254,3,0.120194,-0.001512
Bedrooms,0.012770,0.003754,0.013809,3,0.034282,-0.008741
Sold On,0.000288,0.000944,0.324912,3,0.005695,-0.005119


In [16]:
test

,Sold Price,Sold On,Type,Year built,Bedrooms,Bathrooms,Heating,Cooling,Parking
1,6.322220,2021-02-25,SingleFamily,1951,3,3.0,Central,"Central Air, Dual","Driveway, Driveway - Brick"
5,6.114278,2021-02-24,Townhouse,1966,3,3.0,Central,None,Garage - Attached
40,6.210854,2021-02-17,SingleFamily,1958,4,2.0,"Natural Gas, Fireplace(s), Forced Air",None,Driveway
53,4.562305,2021-02-25,MobileManufactured,1975,2,2.0,Heat Pump,Heat Pump,Attached Carport
58,5.966142,2021-02-18,SingleFamily,1977,3,3.0,"Natural Gas, Central",Central Air,"Driveway, Concrete, Garage, Garage - Two Door"
...,...,...,...,...,...,...,...,...,...
164894,6.076641,2021-02-19,SingleFamily,1995,6,5.0,Central,Central Air,Garage - Attached
164897,5.937017,2021-02-18,SingleFamily,1941,3,2.0,Central,No Data,"Concrete, RV Access/Parking"
164920,6.359625,2021-02-23,SingleFamily,1985,5,5.0,Central,Central Air,"Direct Access, Driveway, Concrete, Driveway Up..."
164932,5.447160,2021-02-17,Condo,1993,2,2.0,Central,Central Air,Parking Space


### 1.1 Results:
By adding new columns it appears that the RMSLE has lowered from the 0.26 to 0.24. However, it appears that the other columns added have a 0 importance as features.

I believe the reason for this is because the columns have no numerical value, so following this I want to try converting some of the data.

## 1.2 Adding Even More Features
I wanted to see what happens if I add even more features. I'm simply adding more columns from `data` into `df`

In [ ]:
# Redefine df
df = data[['Sold Price', 'Sold On', 'Type', 'Year built', 'Bedrooms', 'Bathrooms', 
           'Heating', 'Cooling', 'Parking', 'Garage', 'Water', 'Summary']].copy()

# Establish prediction target and Convert sold price to numerical data.
c = 'Sold Price'
if c in df.select_dtypes('object').columns:
    df.loc[:,c] = np.log10(
            pd.to_numeric(df[c].replace(r'[$,-]', '', regex=True)) + 1)

# Removing outliers in sold price
df = df[(df['Sold Price'] >= 4 ) & (df['Sold Price'] <= 8 )] # prices between 10^4 and 10^8

# Get train, test sets
test_start, test_end = pd.Timestamp(2021, 2, 15), pd.Timestamp(2021, 3, 1)
train_start = pd.Timestamp(2021, 1, 1) # you can change the start time stamp to include more training 
df['Sold On'] = pd.to_datetime(df['Sold On'], errors='coerce')
train = df[(df['Sold On'] >= train_start) & (df['Sold On'] < test_start)]
test = df[(df['Sold On'] >= test_start) & (df['Sold On'] < test_end)]
train.shape, test.shape

# Evaluate Model
eval_model(train, test)

### 1.2 Results:
RMSLE Reduced from 0.24 to 0.2! Big leap from Section 1.1. However, runtime was considerably longer.

## 2.1 Converting Categorical Data to Numerical
In this particular situation, I want to try converting the `Parking` column as I think that having no parking is a huge factor in pricing. Any parking is better than no parking. Thus, I will change '0 spaces' to 0 and anything else as 1.

In [ ]:
# To minimize runtime, I am going to stick with the data from section 1.1
# Redefine df to add Heating, Cooling, Summary
df = data[['Sold Price', 'Sold On', 'Type', 'Year built', 'Bedrooms', 'Bathrooms', 
           'Heating', 'Cooling', 'Parking']].copy()

# Establish prediction target and Convert sold price to numerical data.
c = 'Sold Price'
if c in df.select_dtypes('object').columns:
    df.loc[:,c] = np.log10(
            pd.to_numeric(df[c].replace(r'[$,-]', '', regex=True)) + 1)

# Removing outliers in sold price
df = df[(df['Sold Price'] >= 4 ) & (df['Sold Price'] <= 8 )] # prices between 10^4 and 10^8


In [ ]:
# See column before changing
print(df.loc[:, 'Parking'])
print("\n")

# Function to determine whether 0 parking spaces or not
def parking_to_numeric(x):
    if x=='0 spaces': return 0
    elif x==0: return 0
    else : return 1

# Converting 'Parking' to numerical data in dataframe
df['Parking'] = df['Parking'].apply(parking_to_numeric)

# See column after changing
print(df.loc[:, 'Parking'])

**Evaluating Model**

In [ ]:
# Get train, test sets
test_start, test_end = pd.Timestamp(2021, 2, 15), pd.Timestamp(2021, 3, 1)
train_start = pd.Timestamp(2021, 1, 1) # you can change the start time stamp to include more training 
df['Sold On'] = pd.to_datetime(df['Sold On'], errors='coerce')
train = df[(df['Sold On'] >= train_start) & (df['Sold On'] < test_start)]
test = df[(df['Sold On'] >= test_start) & (df['Sold On'] < test_end)]
train.shape, test.shape

# Evaluate Model
eval_model(train, test)

### 2.1 Results:
Section 1.1 RMSLE = 0.2406 \
Section 2.1 RMSLE = 0.2497 \
Thus the RMSLE slightly increases using my encoding, not ideal.

## 2.2 Converting to Numerical Data using Binary Encoding
In this section, I want to use a library called `category_encoders` to Binary encode `Parking`. I originally wanted to use One-hot, but the encoder would result in a dataframe that overlaps the memory.

In [ ]:
# Reset df to section 1.1 data
# Redefine df to add Heating, Cooling, Summary
df = data[['Sold Price', 'Sold On', 'Type', 'Year built', 'Bedrooms', 'Bathrooms', 
           'Heating', 'Cooling', 'Parking']].copy()

# Establish prediction target and Convert sold price to numerical data.
c = 'Sold Price'
if c in df.select_dtypes('object').columns:
    df.loc[:,c] = np.log10(
            pd.to_numeric(df[c].replace(r'[$,-]', '', regex=True)) + 1)

# Removing outliers in sold price
df = df[(df['Sold Price'] >= 4 ) & (df['Sold Price'] <= 8 )] # prices between 10^4 and 10^8


In [ ]:
# Uncomment line below if not installed
#pip install category_encoders
import category_encoders as ce

# Create object of encoder
ce_be = ce.BinaryEncoder(cols=['Parking'])
df = ce_be.fit_transform(df)
df

In [ ]:
# Get train, test sets
test_start, test_end = pd.Timestamp(2021, 2, 15), pd.Timestamp(2021, 3, 1)
train_start = pd.Timestamp(2021, 1, 1) # you can change the start time stamp to include more training 
df['Sold On'] = pd.to_datetime(df['Sold On'], errors='coerce')
train = df[(df['Sold On'] >= train_start) & (df['Sold On'] < test_start)]
test = df[(df['Sold On'] >= test_start) & (df['Sold On'] < test_end)]
train.shape, test.shape

# Evaluate Model
eval_model(train, test)

### 2.2 Results:
Section 1.1 RMSLE = 0.2406 \
Section 2.1 RMSLE = 0.2497 \
Section 2.2 RMSLE = 0.2469 

Using binary encoding still results in a deprecated RMSLE.\
However, binary encoding still has a higher RMSLE than Section 2.1, but only \
by a small amount

## 3. Data Cleaning - Impute Data with Most Frequent Method
I was hoping to also convert `Heating` and `Cooling` to numerical data, but I noticed that it contained a lot of "No Data" values. Thus I am going to clean them first.

In [ ]:
# Reset df to section 1.1 data
# Redefine df to add Heating, Cooling, Summary
df = data[['Sold Price', 'Sold On', 'Type', 'Year built', 'Bedrooms', 'Bathrooms', 
           'Heating', 'Cooling', 'Parking']].copy()

# Establish prediction target and Convert sold price to numerical data.
c = 'Sold Price'
if c in df.select_dtypes('object').columns:
    df.loc[:,c] = np.log10(
            pd.to_numeric(df[c].replace(r'[$,-]', '', regex=True)) + 1)

# Removing outliers in sold price
df = df[(df['Sold Price'] >= 4 ) & (df['Sold Price'] <= 8 )] # prices between 10^4 and 10^8


In [ ]:
# I want to drop the column if more than 70% of the data is "No Data"
print("Heating: \n")
display(df['Heating'].value_counts())
print("\nCooling: \n")
display(df['Cooling'].value_counts())

In [ ]:
# Also have to check for NaN values in the dataframe
Heating_NaN_Count = df['Heating'].isna().sum()
Cooling_NaN_Count = df['Cooling'].isna().sum()

print("Heating: ")
print((Heating_NaN_Count+36222) / len(df))

print("\nCooling: ")
print((Cooling_NaN_Count+51691) / len(df))

As seen from these results, we don't have to drop these variables because 'No Data' does not consist of 70% of values in these columns. Thus, we can try to impute using an `sklearn` function.

**It is important to note that the impute strategy I'll use is `Most Frequent`**

In [ ]:
from sklearn.impute import SimpleImputer

# Using the imputer on Heating
imputer = SimpleImputer(missing_values='No Data', strategy='most_frequent')
df.Heating = imputer.fit_transform(df['Heating'].values.reshape(-1,1))[:, 0]

# Using the imputer on Cooling
imputer = SimpleImputer(missing_values='No Data', strategy='most_frequent')
df.Cooling = imputer.fit_transform(df['Cooling'].values.reshape(-1,1))[:, 0]

# Do again for 'NaN' values
# Using the imputer on Heating
imputer = SimpleImputer(missing_values=None, strategy='most_frequent')
df.Heating = imputer.fit_transform(df['Heating'].values.reshape(-1,1))[:, 0]

# Using the imputer on Cooling
imputer = SimpleImputer(missing_values=None, strategy='most_frequent')
df.Cooling = imputer.fit_transform(df['Cooling'].values.reshape(-1,1))[:, 0]

# Double checking that there are no 'No Data' values
display(df['Heating'].value_counts())
display(df['Cooling'].value_counts())

In [ ]:
# Get train, test sets
test_start, test_end = pd.Timestamp(2021, 2, 15), pd.Timestamp(2021, 3, 1)
train_start = pd.Timestamp(2021, 1, 1) # you can change the start time stamp to include more training 
df['Sold On'] = pd.to_datetime(df['Sold On'], errors='coerce')
train = df[(df['Sold On'] >= train_start) & (df['Sold On'] < test_start)]
test = df[(df['Sold On'] >= test_start) & (df['Sold On'] < test_end)]
train.shape, test.shape

# Evaluate Model
eval_model(train, test)

### 3. Results:
Section 1.1 RMSLE: 0.2406 \
Section 3 RMSLE: 0.2414 \
Imputing data hardly affected the RMSLE, however I did notice that this section consistently results in an incredibly small increase in RMSLE compared to section 1.1.

## 4. Increasing Examples
I want to increase the training dataset to include data from 2019 to 2021 instead of just 2021 data.

In [ ]:
# Redefine df
df = data[['Sold Price', 'Sold On', 'Type', 'Year built', 'Bedrooms', 'Bathrooms', 
           'Heating', 'Cooling', 'Parking']].copy()

# Establish prediction target and Convert sold price to numerical data.
c = 'Sold Price'
if c in df.select_dtypes('object').columns:
    df.loc[:,c] = np.log10(
            pd.to_numeric(df[c].replace(r'[$,-]', '', regex=True)) + 1)

# Removing outliers in sold price
df = df[(df['Sold Price'] >= 4 ) & (df['Sold Price'] <= 8 )] # prices between 10^4 and 10^8

# Get train, test sets
test_start, test_end = pd.Timestamp(2021, 2, 15), pd.Timestamp(2021, 3, 1)
train_start = pd.Timestamp(2019, 1, 1) # you can change the start time stamp to include more training 
df['Sold On'] = pd.to_datetime(df['Sold On'], errors='coerce')
train = df[(df['Sold On'] >= train_start) & (df['Sold On'] < test_start)]
test = df[(df['Sold On'] >= test_start) & (df['Sold On'] < test_end)]
train.shape, test.shape

# Evaluate Model
eval_model(train, test)

### 4. Results:
Section 1.1 RMSLE = 0.2406 \
Section 4 RMSLE = 0.2390 \
By increasing the dataset to include training data from 2019 and 2020 as well, the RMSLE decreases by just a little bit, about 0.0016 less. 

## EXTRA:
Trying to get the RMSLE as small as I possibly could.
* Include Summary and Zip, noticable increase by adding this data.
* Also adding other data such as High School Score and etc.
* Binary encoding of Amenities included and Type
* Impute year built, Last Sold Price, and High School Score
* Increase examples for training data to 2020

In [ ]:
# Placing this here so I don't have to rerun from the top.
import pandas as pd
import numpy as np

def rmsle(y_hat, y):
    # we already used log prices before, so we only need to compute RMSE
    return sum((y_hat - y)**2 / len(y))**0.5

def eval_model(train, test):
    from autogluon.tabular import TabularPredictor
    label = 'Sold Price'    
    predictor = TabularPredictor(label=label).fit(train, hyperparameters={'GBM':{}})
    preds = predictor.predict(test.drop(columns=[label]))
    print(rmsle(preds, test[label]))

data = pd.read_feather('house_sales.ftr')

In [ ]:
# DELETEME
def none_count(df, col_name):
    display(df[col_name].value_counts())
    display(df[col_name].isna().sum())

In [ ]:
# Redefine df
df = data[['Sold Price', 'Sold On', 'Type', 'Year built', 'Bedrooms', 'Bathrooms', 
           'Summary', 'Zip', 'Last Sold Price', 'High School Score', 'Amenities included']].copy()

# Establish prediction target and Convert sold price to numerical data.
c = 'Sold Price'
if c in df.select_dtypes('object').columns:
    df.loc[:,c] = np.log10(
            pd.to_numeric(df[c].replace(r'[$,-]', '', regex=True)) + 1)
    
# Do same for Last Sold Price
c = 'Last Sold Price'
if c in df.select_dtypes('object').columns:
    df.loc[:,c] = np.log10(
            pd.to_numeric(df[c].replace(r'[$,-]', '', regex=True)) + 1)

# Removing outliers in sold price
df = df[(df['Sold Price'] >= 4 ) & (df['Sold Price'] <= 8 )] # prices between 10^4 and 10^8

In [ ]:
# Data Cleaning
from sklearn.impute import SimpleImputer
from numpy import nanmedian

# Cleaning 'Year built' by imputing most frequent
imputer = SimpleImputer(missing_values='No Data', strategy='most_frequent')
df['Year built'] = imputer.fit_transform(df['Year built'].values.reshape(-1,1))[:, 0]

# Cleaning Last Sold Price
lsp_median = nanmedian(df['Last Sold Price'])
imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=lsp_median)
df['Last Sold Price'] = imputer.fit_transform(df['Last Sold Price'].values.reshape(-1,1))[:, 0]

# Cleaning High School Score
imputer = SimpleImputer(missing_values=None, strategy='most_frequent')
df['High School Score'] = imputer.fit_transform(df['High School Score'].values.reshape(-1,1))[:, 0]

df

In [ ]:
# Binary Encoding
import category_encoders as ce

# Create object of encoder, encode Amenities and Type
ce_be = ce.BinaryEncoder(cols=['Amenities included', 'Type'])
df = ce_be.fit_transform(df)

df

In [ ]:
# Get train, test sets
test_start, test_end = pd.Timestamp(2021, 2, 15), pd.Timestamp(2021, 3, 1)
train_start = pd.Timestamp(2020, 1, 1) # you can change the start time stamp to include more training 
df['Sold On'] = pd.to_datetime(df['Sold On'], errors='coerce')
train = df[(df['Sold On'] >= train_start) & (df['Sold On'] < test_start)]
test = df[(df['Sold On'] >= test_start) & (df['Sold On'] < test_end)]
train.shape, test.shape

# Evaluate Model
eval_model(train, test)


In summary, I couldn't do it. It's hard!